Author: Shailendra Chalasani
Intention: Supervised Learning Likes to dislikes ratio - Decision Trees

In [1]:
import gzip
import json
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import pandas as pd
from collections import Counter, defaultdict
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df_video_dislikes = pd.read_csv(f"../../data/cleaned_datasets/df_video_dislikes.csv")
print(len(df_video_dislikes))
df_video_dislikes.dropna(inplace=True)
print(len(df_video_dislikes))

37422
37264


In [3]:
df_video_dislikes.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'video_id', 'title', 'published_at',
       'view_count', 'likes_count', 'dislikes_count', 'comment_count', 'tags',
       'description', 'comments', 'topic1', 'topic2', 'topic3'],
      dtype='object')

In [4]:
# df_video_dislikes = df_video_dislikes[['title','view_count','likes_count','dislikes_count','comment_count','tags','description','comments']]
df_video_dislikes = df_video_dislikes[['view_count','likes_count','dislikes_count','comment_count','topic1','topic2','topic3']]
# df_video_dislikes['likes_dislikes_ratio'] = df_video_dislikes['likes_count']/df_video_dislikes['dislikes_count']
df_video_dislikes

,view_count,likes_count,dislikes_count,comment_count
0,1048888,19515,226,1319
1,15352638,359277,7479,18729
2,925281,11212,401,831
3,2641597,39131,441,3745
4,715724,32887,367,1067
...,...,...,...,...
37417,120408275,2180780,35871,81360
37418,2841917,20759,1049,2624
37419,13346678,720854,4426,90616
37420,252057,19198,1234,1471


In [5]:
df_stats = pd.read_csv('../data/cleaned_datasets/df_video_stats.csv')
df_stats.dropna(inplace=True)
test_X = df_stats[['view_count','likes_count','comment_count']]

In [6]:
from sklearn.preprocessing import StandardScaler

X = df_video_dislikes[['view_count','likes_count','comment_count']]
y =df_video_dislikes['dislikes_count']

# train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=0)

scaler = StandardScaler().fit(X)
X_train = scaler.transform(X)
X_test = scaler.transform(test_X)

In [10]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# Create the DecisionTreeRegressor object
decision_tree = DecisionTreeRegressor()

# Define the parameter grid for grid search
param_grid = {
    'max_depth': [None, 3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, scoring='r2', cv=5)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y)

# Get the mean and standard deviation of the R2 scores
r2_scores = grid_search.cv_results_['mean_test_score']
r2_mean = np.mean(r2_scores)
r2_std = np.std(r2_scores)

# Get the best parameters
best_params = grid_search.best_params_

print("Mean R2 Score:", r2_mean)
print("Standard Deviation of R2 Score:", r2_std)
print("Best Parameters:", best_params)


Mean R2 Score: 0.37968459748265304
Standard Deviation of R2 Score: 0.13875787471295195
Best Parameters: {'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 5}


In [11]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

# Create the KNeighborsRegressor object
knn = KNeighborsRegressor()

# Define the parameter grid for grid search
param_grid = {
    'n_neighbors': [3, 5, 7,9,11,13,15,17,19,21,23,25],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, scoring='r2', cv=5)

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y)

# Get the mean and standard deviation of the R2 scores
r2_scores = grid_search.cv_results_['mean_test_score']
r2_mean = np.mean(r2_scores)
r2_std = np.std(r2_scores)

# Get the best parameters
best_params = grid_search.best_params_

print("Mean R2 Score:", r2_mean)
print("Standard Deviation of R2 Score:", r2_std)
print("Best Parameters:", best_params)


Mean R2 Score: 0.5546539404859884
Standard Deviation of R2 Score: 0.013687287083409765
Best Parameters: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'uniform'}


In [12]:
# from sklearn.model_selection import GridSearchCV, cross_val_score
# from sklearn.ensemble import GradientBoostingRegressor
# import numpy as np
#
# # Create a GradientBoostingRegressor model
# gbr = GradientBoostingRegressor()
#
# # Define the parameter grid for GridSearchCV
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.1, 0.05, 0.01],
#     'max_depth': [3, 5, 7]
# }
#
# # Create the GridSearchCV object
# grid_search = GridSearchCV(estimator=gbr, param_grid=param_grid, scoring='r2', cv=5)
#
# # Fit the GridSearchCV object to the data
# grid_search.fit(X_train, y)
#
# # Get the mean and standard deviation of the R2 scores
# r2_scores = grid_search.cv_results_['mean_test_score']
# r2_mean = np.mean(r2_scores)
# r2_std = np.std(r2_scores)
#
# # Get the best parameters
# best_params = grid_search.best_params_
#
# print("Mean R2 Score:", r2_mean)
# print("Standard Deviation of R2 Score:", r2_std)
# print("Best Parameters:", best_params)

# Mean R2 Score: 0.4940608547485703
# Standard Deviation of R2 Score: 0.03778774606630465
# Best Parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}

Mean R2 Score: 0.4940608547485703
Standard Deviation of R2 Score: 0.03778774606630465
Best Parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}


In [ ]:
# knn regressor has the best r2 score

In [16]:
# Train the KNeighborsRegressor model
knn = KNeighborsRegressor(n_neighbors=5, metric='manhattan', weights='uniform')
knn.fit(X_train, y)
predicted = knn.predict(X_test)
predicted,np.round(predicted)

(array([ 160. ,  950.8, 1512.6, ...,  144. ,   79.4,   86.4]),
 array([ 160.,  951., 1513., ...,  144.,   79.,   86.]))

In [14]:
df_stats

,Unnamed: 0,title,video_id,published_at,likes_count,comment_count,view_count,comments
0,0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,2022-08-23,3407.0,672.0,135612.0,Let's not forget that Apple Pay in 2014 requir...
1,1,The most EXPENSIVE thing I own.,b3x28s61q3c,2022-08-24,76779.0,4306.0,1758063.0,"Wow, you really went to town on the PSU test r..."
2,2,My New House Gaming Setup is SICK!,4mgePWWCAmA,2022-08-23,63825.0,3338.0,1564007.0,Linus!!! Just turn the key lights 180 and bou...
3,3,My New House Gaming Setup is SICK!,4mgePWWCAmA,2022-08-23,63825.0,3338.0,1564007.0,Linus!!! Just turn the key lights 180 and bou...
4,4,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,2022-08-23,71566.0,1426.0,922918.0,Unstoppable experiments with liquid nitrogen 🎉...
...,...,...,...,...,...,...,...,...
1874,1874,Live Day 1- Introduction To Machine Learning A...,4UJelID_ICw,2022-02-01,1861.0,184.0,37959.0,7:15 AI vs ML vs DL vs DS\n17:00 Machine Learn...
1875,1875,Complete Roadmap for Machine Learning | ML Roa...,T4MLrtOKPjY,2021-05-11,9615.0,648.0,319957.0,Do you want to learn technology from me? Check...
1876,1876,Should You Learn Machine Learning?,AO6urf07KjE,2021-06-14,10259.0,416.0,386360.0,I would also like to point out that actually *...
1877,1877,Todos podemos aprender Machine learning,7ClLKBUvmRk,2017-10-08,2981.0,72.0,431421.0,"Me resultó tan inspirador conocerte, Haydé! Me..."


In [19]:
df_stats['dislikes_count']=np.round(predicted)
print(df_stats)
df_stats.to_csv('../data/cleaned_datasets/df_stats_with_predicted_dislikes.csv')

      Unnamed: 0                                              title   
0              0  Apple Pay Is Killing the Physical Wallet After...  \
1              1                    The most EXPENSIVE thing I own.   
2              2                 My New House Gaming Setup is SICK!   
3              3                 My New House Gaming Setup is SICK!   
4              4  Petrol Vs Liquid Nitrogen | Freezing Experimen...   
...          ...                                                ...   
1874        1874  Live Day 1- Introduction To Machine Learning A...   
1875        1875  Complete Roadmap for Machine Learning | ML Roa...   
1876        1876                 Should You Learn Machine Learning?   
1877        1877            Todos podemos aprender Machine learning   
1878        1878  Andrew Ng: Deep Learning, Education, and Real-...   

         video_id published_at  likes_count  comment_count  view_count   
0     wAZZ-UWGVHI   2022-08-23       3407.0          672.0    135612.0  \

In [9]:
from sklearn.inspection import permutation_importance

# Train the DecisionTreeRegressor model
dtree = DecisionTreeRegressor(min_samples_leaf=3,min_samples_split=2)
dtree.fit(train_X, train_y)

# Calculate permutation importances
result = permutation_importance(dtree, test_X, test_y, n_repeats=10)

# Get the feature importances and their indices
importances = result.importances_mean
indices = np.argsort(importances)

# Print the feature importances
for f in range(train_X.shape[1]):
    print(f"{train_X.columns[indices[f]]}: {importances[indices[f]]}")

comment_count: -0.00020769192181512296
view_count: -7.233538654088933e-05
likes_count: 21.119865924226037
dislikes_count: 21.173016448613225


In [10]:
from sklearn.model_selection import cross_val_score

# Define the range of values for the number of min samples leaf
min_samp_leaf = [1, 3, 5,7]

# Perform sensitivity analysis by iterating over different values of neighbors
for k in min_samp_leaf:
    model = DecisionTreeRegressor(min_samples_leaf=k,min_samples_split=2)

    # Evaluate the model using cross-validation and the desired metric (e.g., mean squared error)
    scores = cross_val_score(model, train_X, train_y, cv=5, scoring='neg_mean_squared_error')

    # Calculate the mean and standard deviation of the scores
    print(scores)
    mean_score = -scores.mean()  # Since scoring='neg_mean_squared_error', we take the negative of the mean score
    std_score = scores.std()

    # Print the results
    print(f"Min samples Leaf: {k}, Mean MSE: {mean_score:.5f}, Standard Deviation: {std_score:.5f}")

[-384.17433389  -84.45907651 -646.99120597 -213.7592489  -171.66825701]
Min samples Leaf: 1, Mean MSE: 300.21042, Standard Deviation: 198.92330
[-277.75907187  -90.75448836 -556.58138925 -265.0219555  -132.43678979]
Min samples Leaf: 3, Mean MSE: 264.51074, Standard Deviation: 163.16605
[-386.09008453 -196.44110644 -593.71937449 -187.36329835 -160.85246965]
Min samples Leaf: 5, Mean MSE: 304.89327, Standard Deviation: 165.12805
[-325.80037195 -169.5030596  -640.2807116  -251.43239469 -212.60528344]
Min samples Leaf: 7, Mean MSE: 319.92436, Standard Deviation: 168.22474


In [16]:
predicted = dtree.predict(test_X)

In [17]:
# MSE is used to evaluate the performance of estimators. A lower MSE indicates that the estimator is more accurate. MSE can also be used to compare the performance of different estimators.
mse = mean_squared_error(test_y, predicted)
mse

118.04314123005595

In [18]:
#In statistics, the coefficient of determination, or R-squared, is a measure of how well a model fits a set of data. It is calculated as the ratio of the variance explained by the model to the total variance in the data.
r_squared = r2_score(test_y, predicted)
r_squared

# This means that the linear regression model explains 67.24% of the variance in the data.

0.9789952058281683

In [19]:
from sklearn.metrics import mean_absolute_error

#If the MAE is 0, this means that the predicted values are perfectly accurate.
# Calculate the accuracy score
mae = mean_absolute_error(test_y, predicted)
mae

2.7604696786453378

In [20]:
df = pd.DataFrame()
df['likes_views_ratio']= test_y
df['Predicted']=predicted
df['Error']=df['likes_views_ratio']-df['Predicted']
df

,likes_views_ratio,Predicted,Error
36171,15.440281,15.608035,-0.167754
34553,44.593028,43.442532,1.150495
21934,24.774297,24.711366,0.062931
14719,19.935759,19.237536,0.698223
20114,193.395564,194.085684,-0.690121
...,...,...,...
2853,32.296875,31.790551,0.506324
5261,22.917402,22.515803,0.401599
10130,30.067073,30.655100,-0.588027
8195,239.596914,248.200980,-8.604066
